<a href="https://colab.research.google.com/github/akuma527/IProjects/blob/master/I%26D_Whale/Baseline_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
%cd /content/drive/My Drive/Colab Notebooks/I&D_Whale/data/

/content/drive/My Drive/Colab Notebooks/I&D_Whale/data


In [0]:
import numpy as np
import pandas as pd

In [0]:
import pickle
with open('Fruits_ORG.obj', 'rb') as fp:
    banana = pickle.load(fp)

In [84]:
banana.head()

,Image,Id,fil_path,type,vector
0,PM-WWA-20050416-037.jpg,-1,train/-1/PM-WWA-20050416-037.jpg,train,"[[-1.0057027, -2.0405624, 0.07455979, -0.06999..."
1,PM-WWA-20050513-229.jpg,-1,train/-1/PM-WWA-20050513-229.jpg,train,"[[-0.19369806, -1.4581945, -0.030819146, 0.290..."
2,PM-WWA-20050513-243.jpg,-1,train/-1/PM-WWA-20050513-243.jpg,train,"[[-1.0048306, -1.2039244, 0.33983347, -1.11833..."
3,PM-WWA-20050520-134.jpg,-1,train/-1/PM-WWA-20050520-134.jpg,train,"[[-0.9010497, -1.292481, -0.53028154, -0.81526..."
4,PM-WWA-20050612-047.jpg,-1,train/-1/PM-WWA-20050612-047.jpg,train,"[[-0.43430337, -1.1216176, 0.49975923, -0.4018..."


In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(0.95)

In [0]:
x = np.concatenate(banana['vector'])

In [0]:
x.shape

(5347, 4096)

In [0]:
X = pca.fit_transform(x)

In [0]:
X.shape

(5347, 570)

In [0]:
dpca = pd.DataFrame(np.concatenate(banana['vector'].to_numpy()))

In [0]:
# dpca = pd.DataFrame(X)

In [94]:
dpca.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-1.005703,-2.040562,0.074560,-0.069998,-0.339858,-0.977509,0.318720,0.092042,-1.146222,0.095260,-1.181991,-0.108332,0.437209,-0.388694,-0.586397,-0.294738,-0.898668,0.140550,-1.211513,-0.908340,-0.535612,0.525601,-0.424789,0.395557,-0.578176,-0.706451,-0.683973,0.513075,-1.135707,-0.168595,0.054922,0.326299,-0.347935,-0.996843,-0.409222,-0.108676,1.077601,-0.758491,-0.230459,-0.862143,...,-1.246361,-0.629989,-0.415247,-0.390340,-0.859891,-1.026045,0.314088,0.260561,-0.318980,0.247871,-0.740547,-0.655837,1.043618,-1.237228,0.705620,0.348582,0.121112,-0.172758,-0.980211,-0.550950,0.627780,-1.604290,-0.370951,-0.974268,-0.298696,0.177567,-0.472354,-0.576754,-0.380234,-0.844706,-0.870705,-0.098448,-0.552447,-0.417900,-1.437631,0.070975,-0.254945,-0.802498,0.099809,-0.336929
1,-0.193698,-1.458194,-0.030819,0.290121,-1.137528,-0.845752,-0.618803,0.182338,-0.516445,0.953905,-0.606780,0.774800,0.228522,-0.296537,-0.350994,-0.389586,-1.068802,-0.047363,-0.785924,-0.298139,-0.261698,0.146701,-0.510146,0.406318,-0.708384,-0.043963,-0.853156,0.408980,-0.694745,-0.294449,-0.003945,0.008390,-0.020875,-1.017562,0.301659,-0.896920,-0.045111,-0.618830,0.282574,-1.046139,...,-0.655824,-0.172530,-0.855194,-0.323745,-0.472007,-0.200471,-0.031003,-0.514225,-0.150909,0.121413,-1.530711,-0.582412,0.637498,-1.714981,0.853332,0.199783,0.391029,-0.865447,-0.952114,0.531189,-0.097069,-0.706211,0.152661,-1.650940,0.912691,-0.590537,-0.421848,-0.559894,-0.163977,0.080422,-0.742440,-0.389644,-0.722660,-0.615074,-0.940163,-0.633879,-0.413949,-0.624646,0.316959,-0.088169
2,-1.004831,-1.203924,0.339833,-0.000011,-0.509310,-0.799318,-0.180213,0.465788,-0.274171,0.321852,-1.011817,0.830478,0.220456,-0.123192,-1.128956,0.477551,-0.225531,-0.009317,-0.832477,-0.406236,-0.650880,-0.282148,-0.741215,0.234141,-0.915255,-0.518611,-0.476537,0.737988,-0.748388,0.018732,-0.330594,0.068055,0.079453,-1.100628,0.384699,-0.609849,-0.104092,0.030313,-0.452016,-0.277236,...,-0.929159,-0.020317,-0.347971,0.009816,-0.683409,0.181636,0.289281,-0.447210,-0.093164,0.213312,-1.106170,-0.287757,0.924939,-1.063456,0.152311,0.741307,0.094977,-0.139617,-0.468608,0.061738,0.066513,-0.697538,0.633666,-0.532701,0.518765,0.212038,-0.787223,-0.502137,-0.598850,0.316705,-0.873665,0.398532,-0.708754,-0.523308,-1.576427,0.034030,0.069727,-0.895598,-0.260377,-0.046986
3,-0.901050,-1.292481,-0.530282,-0.815261,-0.350259,-1.447958,-0.308802,0.110876,-0.751444,-0.316462,-1.519204,0.021437,0.670402,-0.685201,-0.595495,0.005015,-0.388734,-0.997316,-0.188119,-0.457716,-1.232513,0.109794,-0.751745,0.163439,-0.412095,-0.157807,-1.049848,0.521249,-0.854807,-0.431983,-0.079684,-0.503534,-0.762007,-1.369560,-0.427786,-1.076604,-0.097862,-1.104036,-0.612164,-0.632002,...,-0.959422,-0.862727,-1.239127,-0.225197,-0.526487,-0.897159,-0.578658,-0.415344,-0.401070,0.096086,-0.709880,-1.151952,0.733642,-1.364059,0.677339,0.668924,-0.335045,0.271352,-1.643885,-0.070536,-0.566989,-0.863420,-0.026974,-1.263147,-0.045055,-0.429234,-1.209146,-0.776752,-1.261935,-1.240182,-1.069512,-0.220989,-1.380595,-0.437730,-1.245605,-0.522632,0.322174,-1.325427,-0.402670,-0.048166
4,-0.434303,-1.121618,0.499759,-0.401823,-1.930602,-1.181080,-0.598661,0.185432,-1.307527,1.030882,-1.238716,-1.345303,0.861316,-0.296807,-1.564389,0.206569,-0.597026,0.204947,-1.120790,-0.067666,0.318221,0.327958,-0.332818,0.183903,-0.853552,-0.310020,-1.222915,0.978109,-0.420260,-0.625172,-0.146639,0.259795,0.571701,-1.298091,-0.297787,-0.569947,0.284901,0.249034,0.417760,-0.473903,...,-0.960915,-0.101906,-0.028394,-0.773181,-0.787844,0.380052,-0.620677,-0.546078,-0.230741,0.209743,-1.547994,-0.732941,1.013310,-2.205306,1.434098,-0.850388,0

In [0]:
df = pd.merge(banana[['Image']], dpca, left_index=True, right_index=True)

In [0]:
df.set_index('Image', inplace=True)

In [97]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
Image,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PM-WWA-20050416-037.jpg,-1.005703,-2.040562,0.074560,-0.069998,-0.339858,-0.977509,0.318720,0.092042,-1.146222,0.095260,-1.181991,-0.108332,0.437209,-0.388694,-0.586397,-0.294738,-0.898668,0.140550,-1.211513,-0.908340,-0.535612,0.525601,-0.424789,0.395557,-0.578176,-0.706451,-0.683973,0.513075,-1.135707,-0.168595,0.054922,0.326299,-0.347935,-0.996843,-0.409222,-0.108676,1.077601,-0.758491,-0.230459,-0.862143,...,-1.246361,-0.629989,-0.415247,-0.390340,-0.859891,-1.026045,0.314088,0.260561,-0.318980,0.247871,-0.740547,-0.655837,1.043618,-1.237228,0.705620,0.348582,0.121112,-0.172758,-0.980211,-0.550950,0.627780,-1.604290,-0.370951,-0.974268,-0.298696,0.177567,-0.472354,-0.576754,-0.380234,-0.844706,-0.870705,-0.098448,-0.552447,-0.417900,-1.437631,0.070975,-0.254945,-0.802498,0.099809,-0.336929
PM-WWA-20050513-229.jpg,-0.193698,-1.458194,-0.030819,0.290121,-1.137528,-0.845752,-0.618803,0.182338,-0.516445,0.953905,-0.606780,0.774800,0.228522,-0.296537,-0.350994,-0.389586,-1.068802,-0.047363,-0.785924,-0.298139,-0.261698,0.146701,-0.510146,0.406318,-0.708384,-0.043963,-0.853156,0.408980,-0.694745,-0.294449,-0.003945,0.008390,-0.020875,-1.017562,0.301659,-0.896920,-0.045111,-0.618830,0.282574,-1.046139,...,-0.655824,-0.172530,-0.855194,-0.323745,-0.472007,-0.200471,-0.031003,-0.514225,-0.150909,0.121413,-1.530711,-0.582412,0.637498,-1.714981,0.853332,0.199783,0.391029,-0.865447,-0.952114,0.531189,-0.097069,-0.706211,0.152661,-1.650940,0.912691,-0.590537,-0.421848,-0.559894,-0.163977,0.080422,-0.742440,-0.389644,-0.722660,-0.615074,-0.940163,-0.633879,-0.413949,-0.624646,0.316959,-0.088169
PM-WWA-20050513-243.jpg,-1.004831,-1.203924,0.339833,-0.000011,-0.509310,-0.799318,-0.180213,0.465788,-0.274171,0.321852,-1.011817,0.830478,0.220456,-0.123192,-1.128956,0.477551,-0.225531,-0.009317,-0.832477,-0.406236,-0.650880,-0.282148,-0.741215,0.234141,-0.915255,-0.518611,-0.476537,0.737988,-0.748388,0.018732,-0.330594,0.068055,0.079453,-1.100628,0.384699,-0.609849,-0.104092,0.030313,-0.452016,-0.277236,...,-0.929159,-0.020317,-0.347971,0.009816,-0.683409,0.181636,0.289281,-0.447210,-0.093164,0.213312,-1.106170,-0.287757,0.924939,-1.063456,0.152311,0.741307,0.094977,-0.139617,-0.468608,0.061738,0.066513,-0.697538,0.633666,-0.532701,0.518765,0.212038,-0.787223,-0.502137,-0.598850,0.316705,-0.873665,0.398532,-0.708754,-0.523308,-1.576427,0.034030,0.069727,-0.895598,-0.260377,-0.046986
PM-WWA-20050520-134.jpg,-0.901050,-1.292481,-0.530282,-0.815261,-0.350259,-1.447958,-0.308802,0.110876,-0.751444,-0.316462,-1.519204,0.021437,0.670402,-0.685201,-0.595495,0.005015,-0.388734,-0.997316,-0.188119,-0.457716,-1.232513,0.109794,-0.751745,0.163439,-0.412095,-0.157807,-1.049848,0.521249,-0.854807,-0.431983,-0.079684,-0.503534,-0.762007,-1.369560,-0.427786,-1.076604,-0.097862,-1.104036,-0.612164,-0.632002,...,-0.959422,-0.862727,-1.239127,-0.225197,-0.526487,-0.897159,-0.578658,-0.415344,-0.401070,0.096086,-0.709880,-1.151952,0.733642,-1.364059,0.677339,0.668924,-0.335045,0.271352,-1.643885,-0.070536,-0.566989,-0.863420,-0.026974,-1.263147,-0.045055,-0.429234,-1.209146,-0.776752,-1.261935,-1.240182,-1.069512,-0.220989,-1.380595,-0.437730,-1.245605,-0.522632,0.322174,-1.325427,-0.402670,-0.048166
PM-WWA-20050612-047.jpg,-0.434303,-1.121618,0.499759,-0.401823,-1.930602,-1.181080,-0.598661,0.185432,-1.307527,1.030882,-1.238716,-1.345303,0.861316,-0.296807,-1.564389,0.206569,-0.597026,0.204947,-1.120790,-0.067666,0.318221,0.327958,-0.332818,0.183903,-0.853552,-0.310020,-1.222915,0.978109,-0.420260,-0.625172,-0.146639,0.259795,0.571701,-1.298091,-0.297787,-0.569947,0

In [0]:
# df.to_csv('pca_df.csv')

# Similarity Matrix

In [0]:
# df.set_index('Image', inplace=True)
# df.head()

In [0]:
k = pd.merge(banana[['Image', 'type']], df.reset_index(), left_on='Image', right_on='Image')

In [101]:
test = k[k['type']=='test']
test.drop(columns=['type'], inplace=True)
test = test.set_index('Image')
test.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
Image,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PM-WWA-20170321-046.jpg,-1.451752,-1.404735,-0.228510,0.236677,-1.026403,-0.486075,0.009470,-0.125524,-0.705968,0.506156,-1.217933,-0.698443,0.560499,-0.384823,-0.964896,-0.258156,-0.976660,-0.619541,-1.074016,-0.060139,-0.814454,0.805409,0.177404,0.328957,0.645755,-0.093764,-0.778841,0.555445,-0.314127,-1.020971,-0.401576,-0.305862,-0.143923,-0.659677,0.348798,-0.441590,-0.180560,0.004002,0.198676,-0.456349,...,-0.205414,0.330176,0.542670,-0.935553,-0.735015,0.364410,-0.625458,-0.924446,-0.057349,0.155485,-0.806406,0.353661,0.758603,-1.330958,0.194492,-1.138423,0.650802,0.807238,0.139751,-0.004846,-0.989370,-0.874771,-0.013135,-0.990813,-0.319521,-0.705746,-0.851282,-0.883475,-0.487676,-0.782817,-0.773423,-0.755083,-0.870651,0.102721,-0.983739,0.319416,0.208558,-0.580112,-0.394110,0.647171
PM-WWA-20170321-053.jpg,-1.060490,-1.227558,0.406937,0.588267,-1.272455,-0.739962,-0.239763,-0.181962,-0.590463,0.838801,-1.283979,-0.643996,0.521042,-0.084297,-0.871416,0.725586,-0.570539,-0.619145,-1.160428,0.111412,-0.485015,0.945969,0.004434,0.417683,-0.263627,0.171406,-0.725493,0.290658,-0.670974,-0.570559,-0.286713,0.635525,-0.043143,-0.863444,-0.245373,-0.248460,0.024512,-0.238759,0.289294,-0.330301,...,0.213448,-0.505635,0.291479,0.029751,-0.705986,0.132631,-0.557322,-1.268963,-0.112647,0.118411,-0.531978,-0.317181,-0.025671,-0.685435,0.638365,-0.978338,0.185205,-0.175467,0.180100,0.140908,-0.810617,-0.564304,0.060797,-0.895332,0.146861,-0.758664,-0.650827,-0.356032,-0.046719,-0.653844,-0.642808,-0.850923,-0.460066,-0.165683,-0.779849,0.341081,0.381159,-0.761401,-0.255400,0.348217
PM-WWA-20170321-322.jpg,-0.846898,-0.891654,0.230187,-0.203885,-0.372782,-1.503502,-0.218343,0.668493,-0.417522,0.443089,-1.179450,1.100462,0.663722,-0.173734,-0.460439,0.390430,-0.254806,-0.569953,-0.504223,-0.187462,-0.861548,-0.045001,-0.740791,0.156607,-0.774554,-0.366216,-1.428619,0.629972,-0.825826,0.370559,0.032279,-0.255922,-0.500094,-0.994733,0.847021,-0.500181,-0.099190,-0.428093,-0.368763,-0.514865,...,-0.417109,-0.595160,-0.298723,-0.273335,-0.631875,0.164678,-0.145400,-1.089452,0.040899,0.117880,-1.344451,-0.160083,0.419232,-1.155747,0.766195,0.598909,0.984046,0.144431,-1.264776,0.358392,-0.208597,-0.644078,0.360032,-0.531832,0.205837,-0.667422,-0.936611,-0.787372,-0.893058,-0.625713,-0.578703,0.038559,-1.004791,-0.906767,-1.120287,-0.380022,0.940034,-0.930277,-0.177769,0.572285
PM-WWA-20170321-332.jpg,-1.562151,-0.349217,-0.229540,-0.298605,-1.379045,-0.041419,-0.607495,0.259003,-0.367063,0.472438,-2.315762,-0.746887,0.801444,-0.517415,-1.457006,0.153633,-0.921047,-0.719833,-0.804572,0.760730,-0.554689,0.989402,0.182542,0.198425,-0.213099,-0.688407,-1.449830,0.327196,0.010442,-0.704741,-0.203611,0.569000,-0.008034,-0.921121,-0.023415,-0.985950,0.187712,-0.161454,0.360286,-0.053472,...,-0.336600,-0.634703,-0.189365,-0.880330,-0.296752,0.070889,-0.737296,-0.634568,-0.178939,0.159846,-0.641142,-0.130960,0.098376,-1.489823,0.045227,-1.478942,0.204946,0.444988,0.251601,0.451529,-1.640585,-1.112059,-0.085100,-0.864721,0.637079,-0.543698,-1.479533,-0.665906,0.498625,-0.635723,-0.283374,-0.652617,-0.379760,0.085995,-0.907560,-0.175904,0.094069,-1.306006,-0.956501,0.702459
PM-WWA-20170321-344.jpg,-1.410984,-1.359070,-0.818786,-0.436968,-1.239132,-1.008135,-0.126728,0.028478,-0.249752,0.877554,-1.805589,-0.259232,0.885355,-0.662137,-1.722930,1.189824,-0.587820,-1.147941,-1.064201,0.179278,-0.569682,0.580386,-0.158795,0.509719,-0.093651,-0.785557,-1.367803,0.573343,-0.138528,0.386885,-0.279397,0.344655,0.543071,-0.847223,0.220704,-1.226356,-0.040671,-0.410777

In [0]:
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
def similarity_func(u, v):
    return cosine_similarity(u.reshape(1,-1), v.reshape(1,-1))

In [0]:
from tqdm import tqdm_notebook

In [106]:
p = []
for i in tqdm_notebook(test.index):
    for j in df.index:
        p.append(similarity_func(test.loc[i,:].to_numpy(), df.loc[j,:].to_numpy()))

In [0]:
arr = np.reshape(p, (len(test), len(df)))

In [0]:
df = pd.DataFrame(arr, index=test.index, columns=df.index)

In [0]:
k = []
for i in test.index:
    k.append(list(df.loc[i, :].sort_values(ascending=False).head(21).index))

In [0]:
len(k)

808

In [0]:
pd.DataFrame(k).to_csv('pred_vgg.csv',index=False)